# 🚀 GPT-2: Generative Pre-trained Transformer 2

This notebook explores **GPT-2**, a powerful text generation model from OpenAI. GPT-2 uses the **decoder-only** architecture and is trained with **Causal Language Modeling (CLM)**.

## Key Concepts

| Concept | Description |
|:--------|:------------|
| **Architecture** | Decoder-only Transformer (12 layers in small) |
| **Attention** | Causal (unidirectional) - only sees past tokens |
| **Training** | Causal Language Modeling (next token prediction) |
| **Output** | Probability distribution over vocabulary for next token |

## GPT-2 vs BERT Comparison

```text
BERT (Encoder):                GPT-2 (Decoder):
├─ Bidirectional               ├─ Unidirectional (left-to-right)
├─ Masked Language Modeling    ├─ Causal Language Modeling
├─ Good for understanding      ├─ Good for generation
└─ Classification tasks        └─ Text completion tasks
```

---

## 1. Quick Text Generation with Pipeline

The easiest way to use GPT-2 is through Hugging Face's `pipeline` API. This handles tokenization, model inference, and decoding automatically.

**Parameters:**
- `max_length`: Maximum number of tokens to generate
- `num_return_sequences`: How many different completions to generate
- `set_seed`: For reproducible outputs

In [ ]:
import torch
from transformers import pipeline, set_seed

generator = pipeline('text-generation', model='gpt2')
set_seed(42)
generator("I am a nerd and,", max_length=30, num_return_sequences=5)

## 2. Understanding GPT-2 Tokenization

GPT-2 uses **Byte-Pair Encoding (BPE)** tokenization, which is different from BERT's WordPiece:

| Feature | BERT (WordPiece) | GPT-2 (BPE) |
|:--------|:-----------------|:------------|
| Subword prefix | `##` for continuation | `Ġ` for space before |
| Special tokens | `[CLS]`, `[SEP]` | None required |
| Vocabulary size | ~30,000 | ~50,257 |

The `Ġ` symbol represents a space before the token (it's the byte representation of a space).

In [ ]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

prompt = "Criminal Thrillers are my favourite genre of"
inputs = tokenizer(prompt, return_tensors='pt')
inputs

### Visualizing the Tokens

Let's see how the text is split into tokens:

In [ ]:
tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

**Token Breakdown:**

```text
Token         Meaning
─────────────────────────
'C'           Start of "Criminal"
'riminal'     Continuation (no space)
'ĠThr'        Space + "Thr"
'ill'         Continuation
'ers'         Continuation  
'Ġare'        Space + "are"
'Ġmy'         Space + "my"
'Ġfavourite'  Space + "favourite"
'Ġgenre'      Space + "genre"
'Ġof'         Space + "of"
```

---

## 3. GPT-2 for Language Modeling

For text generation, we use `GPT2LMHeadModel` which adds a **language modeling head** on top of the base model.

```text
GPT2LMHeadModel Architecture:

    Input Tokens
         ↓
    Token Embeddings + Position Embeddings
         ↓
    ┌─────────────────────────┐
    │  12× Transformer Decoder │
    │  (Masked Self-Attention) │
    └─────────────────────────┘
         ↓
    Linear Layer (hidden → vocab_size)
         ↓
    Logits (50,257 values per position)
         ↓
    Softmax → Probabilities
         ↓
    Next Token Prediction
```

In [ ]:
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained('gpt2')
outputs = model(**inputs)
outputs

## 4. Understanding the Logits

The model outputs **logits** - raw scores for each token in the vocabulary at each position.

**Key insight:** For next-token prediction, we only care about the logits at the **last position**.

In [ ]:
logits = outputs.logits
logits.shape

**Shape breakdown:** `[1, 10, 50257]`
- `1` = batch size
- `10` = sequence length (10 input tokens)
- `50257` = vocabulary size (probability for each possible next token)

```text
Position:  0   1   2   3   4   5   6   7   8   9
Token:    "C" "rim" "Thr" "ill" "ers" "are" "my" "fav" "genre" "of"
                                                              ↓
                                                         We want this!
                                                         (predict what comes after "of")
```

In [ ]:
last_token_logits = logits[:, -1, :]
last_token_logits

## 5. Predicting the Next Token

To get the predicted next token, we take the **argmax** of the logits (the token with the highest score).

This is called **greedy decoding** - always picking the most likely token.

In [ ]:
next_token_id = torch.argmax(last_token_logits).item()
next_token_id

### Decoding the Token ID

Let's convert this token ID back to text:

In [ ]:
tokenizer.decode(next_token_id, skip_special_tokens=True)

**Result:** The model predicts **" horror"** as the next word!

Full sentence: *"Criminal Thrillers are my favourite genre of **horror**"*

---

## 🔄 The Autoregressive Generation Process

To generate more text, we repeat this process:

```text
Step 1: "Criminal Thrillers are my favourite genre of" → predict "horror"
Step 2: "Criminal Thrillers are my favourite genre of horror" → predict "movies"
Step 3: "Criminal Thrillers are my favourite genre of horror movies" → predict "."
...
```

This is what the `pipeline` does automatically!

---

## 📚 Key Takeaways

| Concept | Description |
|:--------|:------------|
| **BPE Tokenization** | Uses `Ġ` prefix for tokens with leading space |
| **Causal Attention** | Each position only attends to previous positions |
| **Logits** | Raw scores for each vocabulary token |
| **Greedy Decoding** | Pick the token with highest probability |
| **Autoregressive** | Generate one token at a time, using previous as context |

---

## 🚀 Next Steps

- See `BERT.ipynb` for encoder-only model comparison
- See `spam_classification.ipynb` for a practical classification task